In [139]:

import sys,os
sys.path.insert(0, os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(1,'../')
import hashlib
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from pathlib import Path
import pandas as pd
import math
import run_esmfold as ef

In [172]:
base_dir = Path("/hpcfs/fhome/shizhenkun/codebase/RXNRECer/case/fusarium_venenatum/data/")
fasta_path = base_dir / "ncbi_protein.fasta"    # 输入序列文件
    
data_input_fasta_records = ef.get_fasta_with_seq_hash(fasta_path=fasta_path) 
# 设置目标目录
pdb_dir = "/hpcfs/fhome/shizhenkun/codebase/RXNRECer/data/structure/pdb/ncbi/"

# 获取所有.pdb文件的完整路径
existing_pdb_files = list(Path(pdb_dir).rglob("*.pdb"))
existing_pdb_files = [f.stem for f in existing_pdb_files] 
data_input_fasta_records = data_input_fasta_records[~data_input_fasta_records.id.isin(existing_pdb_files)].reset_index(drop=True)
data_input_fasta_records['seqlen']=data_input_fasta_records.sequence.apply(len)
data_input_fasta_records =data_input_fasta_records.sort_values(by="seqlen").reset_index(drop=True)

data_input_fasta_records

,id,seq_hash,description,sequence,seqlen
0,XP_065463034.1,ffeffb18ab720bb93bbb9333a0cc5239cd8cf7f221e412...,RFX DNA-binding domain-containing protein [Fus...,MYSGQWLPGDHAHIHAHPHGHVKDMSGTQMSTDDMLLQAATHMQGG...,768
1,XP_065463032.1,a53acb312742f13b5b998e165279f7d6c0be4396068a7f...,exocyst complex component Sec10-like protein [...,MYRSKTGAANRGSILSTLKATEMLDTKASLPAEILVAILDYLPVAD...,923
2,XP_025586171.2,198874dc0ed3b731a31ed6ce856fa07729f6b8db5592d9...,uncharacterized protein EDB82DRAFT_575363 [Fus...,MSNQILLKNGTVLQHDEKDNIIVLRNTDILISDGRIAEIGKGINKP...,937
3,XP_025584570.1,c68106a2cf8a552da5e755b6a5df18d781847bbf6cc2d3...,uncharacterized protein EDB82DRAFT_574114 [Fus...,MSANNAPQPVKLSLPLEYQQSLFQELRAEDELVVIARGLGLMRLVN...,937
4,XP_065464016.1,bc64dcf9ce90c75777c1791e277752d47976443d797b53...,uncharacterized protein EDB82DRAFT_464487 [Fus...,MAIDTLSLQCKIAIVTGSGRENGIGAGIAIALARNGAAVTIHYVND...,937
...,...,...,...,...,...
186,XP_065463288.1,516d97a6a83683b7ee1e0f4b2909856f07dff286bf9ee8...,uncharacterized protein EDB82DRAFT_305551 [Fus...,MELTGLSIINAQASRQPGPHLLHQLVKPPSQHAALDYMGSSQRADF...,4820
187,XP_025589913.2,2708978dffc00b29d3ef1367e9de917f6a370a8639d6be...,"uncharacterized protein EDB82DRAFT_486324, par...",KKSLASFICADSGAILNDGTEGSSVLAMDDSFRSLAKELEISLSNS...,5427
188,XP_065465813.1,a1cfaf34251330256c664ecb3ca758012f7d11a981f278...,"uncharacterized protein EDB82DRAFT_442080, par...",MGLPWEIITATSAISTLTLAWAHDDWSKSQLLGRFPVLWIAGFAVW...,5470
189,XP_065465618.1,684436277aae93fa730b72b19a01874572682f81845185...,uncharacterized protein EDB82DRAFT_52575 [Fusa...,MAAFEDAVTFLLEGVDLSIRDFPSHSSQDRAAFIKWNSAKPARLES...,8005


In [177]:
tempdir = base_dir / "slurm_temp"               # 临时文件夹用于存放分割后的 fasta 文件
chunk_info = ef.chunk_fasta(df=data_input_fasta_records, sequences_per_chunk=400, outdir=tempdir)

Chunk 1: 167 seqs, min = 768, max = 999, avg = 966.5, partition = qgpu_a40
>XP_025582255.1
MEYLTAVDGRQDLPPTPASFCSHGDSPLNSSYEQLFHLYGLDSSRIEAIKPCTPFQLDMIDCNALDKQSAIGHAVYDVPTDIDISRFALAWKEIVNQTPALRAFAFTSDSGKTSQVILKDSFVFSWMCWSSSSSPDEVVRDEAAAAASGPRCNRFVLLEDMQTKKCQLVWTFSHALVDVTFQQRVLSRVFAAYKHEKDTHRPETPESSDATDTDSQSVSVVSMSCEDNAVSATHFWQTHLNDLNASVFPHLSDHLMVPNPTTTAEHRITFPLSQKALSNSAICRTALSILLSRYTHSDEALFGAVTEQSLPFDKHYLADGTYQTVAPLRVHCQSNLRASDVMDAISSYDDRLGHLAPFGLRDIRNTGDNGSAACDFQTVLLVTDGSHVNNGINGFLQQITESSHFMPCNNRALLLHCQMESSGALLVAYYDHNVIDSLQTTRLLQQFGHLIKCLQSPLDLSSMAEVNLMTEYDRAEIESWNSQPLEVQDTLIHHEMLKAVSHSPTKTAIQAWDGDWTYSELDNVSSRLAVHIKSLGLRAQQAIIPVYFEKSKWVIASMLAVLKSGNAFTLIDPNDPPARTAQVVTQTRATVALTSKLHRETVQKLVGRCVVVDDELLQSVSASDDFSSLTKSQDLAYVIFTSGSTGDPKGIMIEHRAFSSCALKFGASLGINSDTRALQFGTHAFGACLLEIMTTLINGGCVCIPSDDDRMNSIPSFINRYNVNWMMATPSYMGTFSPEDVPGLATLVLVGEQMSSSVNAIWAPKLQLLNGYGQSESSSICFASNMSTEPNNMGRAVGAHSWVIDPNDINRLVPIGAVGELVIESPGIARDYIVPPPPEKSPFFTDIPSWYPANTFPDGAKLYRTGDLARYASDGSIVCLGRIDSQVKIRGQRVELGAIETHLRQQMPD

In [ ]:
data_input_fasta_records